# **Multivariate Gaussian models and Gaussian Mixture Models** 

##  Multivariate Gaussian models

Description: <br> 

A simple classifier using multivariate Gaussian models. Each class in the data is modeled by a single 3D Gaussian distribution. <br>

Two different structures for the covariance matrices are considered for the model, which are:
- Each Gaussian uses a diagonal covariance matrix.
- Each Gaussian uses a full covariance matrix

Model Explanation: <br>

We first start by loading and preprocessing the data. The data is loaded via the read csv
method which render the data in a dataframe. Next, we convert the dataframe into numpy
arrays and separate the labels from the input data. The input data is further normalized for
faster processing and the binary labels are converted to 0, 1 labels for easier processing.


In [ ]:
import pandas as pd

train_data = pd.read_csv("train-gaussian.csv")
test_data = pd.read_csv("test-gaussian.csv")
print(train_data.shape)
print(test_data.shape)

(1891, 4)
(830, 4)


In [ ]:
print(list(train_data.columns.values))
print(list(test_data.columns.values))

['class', ' x1', ' x2', ' x3']
['class', 'x1', 'x2', 'x3']


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Xtrain =  train_data[[' x1', ' x2', ' x3']].to_numpy().astype(np.float16)
Xtrain = MinMaxScaler().fit_transform(Xtrain)
print(Xtrain.shape)
print(type(Xtrain))

ytrain = train_data[['class']].to_numpy()
ytrain = np.where(ytrain == 'A', 0, 1)
ytrain = ytrain.astype(np.float16)
print(ytrain.shape)
print(type(ytrain))

Xtest =  test_data[['x1', 'x2', 'x3']].to_numpy().astype(np.float16)
Xtest = MinMaxScaler().fit_transform(Xtest)
print(Xtest.shape)
print(type(Xtest))

ytest = test_data[['class']].to_numpy()
ytest = np.where(ytest == 'A', 0, 1)
ytest = ytest.astype(np.float16)
print(ytest.shape)
print(type(ytest))

(1891, 3)
<class 'numpy.ndarray'>
(1891, 1)
<class 'numpy.ndarray'>
(830, 3)
<class 'numpy.ndarray'>
(830, 1)
<class 'numpy.ndarray'>


In [ ]:
print(Xtrain[0])
print(ytrain[:10])
print(Xtest[0])
print(ytest[:10])

[0.5977 0.571  0.6875]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[0.5625 0.6216 0.6323]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


Now, to build a binary classifier using multivariate Gaussian models, we build a class called GDA that consists of all model functions and parameters. We initialize the model with the
training data and separate the two classes to model their individual Gaussian distributions.


Next, for each class we compute the mean and covariance matrix for the Gaussian distribution and the diagonal elements of the full covariance matrix are taken to build the
diagonal covariance matrix for each class. We thus complete the training part of the model
and print the mean vectors, covariance vectors, and diagonal vectors for both classes.
Next, we evaluate the model on the test data. The predict function takes in the test data and
the co var mat parameter which takes the full covariance matrix for prediction if set to 0, else
takes the diagonal covariance matrix. Thus, for every point in the test data, its probability is
computed for both the classes, and the point is assigned to the class with a higher probability.
This probability is computed using the MAP decision rule. We call the predict function for
both the full covariance matrix and the diagonal covariance matrix, and compare the accuracy
between the two. We thus obtain the following results for the experiment:

In [ ]:
from sklearn.metrics import accuracy_score

class GDA:
    def __init__(self,train_data,train_label):
  
        self.Train_Data = train_data
        self.Train_Label = train_label
        self.postive_num = 0                                                    
        self.negetive_num = 0                                                   
        postive_data = []                                                       
        negetive_data = [] 
                                                             
        for (data,label) in zip(self.Train_Data,self.Train_Label):
            if label == 1:          
                self.postive_num += 1
                postive_data.append(list(data))
            else:                   
                self.negetive_num += 1
                negetive_data.append(list(data))

        row,col = np.shape(train_data)                                    
        # Positive and negative samples calculated mean vector of the Gaussian distribution
        postive_data = np.array(postive_data)
        negetive_data = np.array(negetive_data)
        postive_data_sum = np.sum(postive_data, 0)
        negetive_data_sum = np.sum(negetive_data, 0)
        self.mu_positive = postive_data_sum*1.0/self.postive_num                
        self.mu_negetive = negetive_data_sum*1.0/self.negetive_num    
        print("Class A mean: ",self.mu_negetive )
        print("Class B mean: ",self.mu_positive )

        # Negative sample mean vector of the Gaussian distribution
        # Compute the covariance matrix of the Gaussian distribution
        
        positive_deta = postive_data-self.mu_positive
        negetive_deta = negetive_data-self.mu_negetive
        self.sigma = []
        self.sigma_diag = []

        for deta in positive_deta:
            deta = deta.reshape(1,col)
            ans = deta.T @ deta
            self.sigma.append(ans)

        for deta in negetive_deta:
            deta = deta.reshape(1,col)
            ans = deta.T @ deta
            self.sigma.append(ans)

        self.sigma = np.array(self.sigma)
        #print(np.shape(self.sigma))
        self.sigma = np.sum(self.sigma,0)
        self.sigma = self.sigma/row
        print("\nFull cov matrix:")
        print(self.sigma)
        self.sigma_diag = np.diag(np.diag(self.sigma))
        print("\nDiagonal cov matrix:")
        print(self.sigma_diag)
        self.mu_positive = self.mu_positive.reshape(1,col)
        self.mu_negetive = self.mu_negetive.reshape(1,col)

    def Gaussian(self, x, mean, cov):
        dim = np.shape(cov)[0]
        # Cov measures of the determinant is zero
        covdet = np.linalg.det(cov + np.eye(dim) * 0.001)
        covinv = np.linalg.inv(cov + np.eye(dim) * 0.001)
        xdiff = (x - mean).reshape((1, dim))
        # Probability Density
        prob = 1.0 / (np.power(np.power(2 * np.pi, dim) * np.abs(covdet), 0.5)) * \
               np.exp(-0.5 * xdiff @ covinv @ xdiff.T)[0][0]
        return prob

    def predict(self,test_data, co_var_mat):
        predict_label = []
        for data in test_data:
          # predict based on full co-variance matrix.
            if co_var_mat == 0:
              class_B_prob = self.Gaussian(data,self.mu_positive,self.sigma)
              class_A_prob = self.Gaussian(data,self.mu_negetive,self.sigma)
            else:
              class_B_prob = self.Gaussian(data,self.mu_positive,self.sigma_diag)
              class_A_prob = self.Gaussian(data,self.mu_negetive,self.sigma_diag)
              
            if class_B_prob >= class_A_prob:
                predict_label.append(1)
            else:
                predict_label.append(0)
        return predict_label

def run_main():

    # GDA results
    gda = GDA(Xtrain,ytrain)

    #Predict for full co-variance matrix
    y_full_pred = gda.predict(Xtest,0)
    print("\nGDA accuracy with full cov matrix:",accuracy_score(ytest,y_full_pred))

    #Predict for diagonal co-variance matrix
    y_diag_pred = gda.predict(Xtest,1)
    print("\nGDA accuracy with diagonal cov matrix:",accuracy_score(ytest,y_diag_pred))

if __name__ == '__main__':
    run_main()

Class A mean:  [0.64449763 0.56028706 0.5430622 ]
Class B mean:  [0.52718675 0.5505319  0.6524823 ]

Full cov matrix:
[[ 0.02126539 -0.00850425 -0.00650579]
 [-0.00850425  0.03082928  0.01286693]
 [-0.00650579  0.01286693  0.03073367]]

Diagonal cov matrix:
[[0.02126539 0.         0.        ]
 [0.         0.03082928 0.        ]
 [0.         0.         0.03073367]]

GDA accuracy with full cov matrix: 0.6566265060240963

GDA accuracy with diagonal cov matrix: 0.6602409638554216


From the results, we observe that the multivariate Gaussian model does not fit the training data very well due to its high dimension and thus requires a more complex model. This can be verified with the accuracies obtained on the test data where the model performs slightly better with the diagonal covariance matrix.


## Gaussian Mixture Models

Description: <br> 

To improve the Gaussian classifier from the multivariate Gaussian model by using a Gaussian Mixture Model to
model each class. <br>

Investigate the Gaussian Mixture Models that have 2, 4, 8, or 16 Gaussian components and determine the best
model configuration in terms of the number of Gaussian components and the covariance
matrix structure (diagonal vs. full) for the given data set.

Model Explanation: <br> 

Here, we implement Gaussian
Mixture Models (GMMs) for the two classes in the dataset. We build a separate GMM for
each class, and cluster each GMM into a given number of clusters. Next, we predict the class
of each data in a given GMM using the MAP rule and evaluate the predictions against their
true labels

We initialize the model using the number of clusters and the number of training epochs. For
faster training, the number of epochs is 1 here for which the model gives sufficiently good
results. Next, we separate the training data according to their classes and initiate the model
training for each class. The training process for each class includes initializing the mean
vector, the covariance vector, and the weights vector using k-means clustering which will be
used as starting points for the GMM. Next, the ’E’ and the ’M’ steps are performed for each
class which update the phi and the weights variables in the ’E’ step, and the mean and the
covariance variables in the ’M’ step. This process continues for a given number of epochs and
return the mean vector, covariance matrix, and the diagonal covariance matrix upon
completion. Thus, we obtain all three variables (mean, covariance, diagonal covariance) for
each cluster in a given class.

Now, to predict the class of each data point in the test data, we first predict the cluster that
each data point belongs to. Once the class labels for each data point are obtained, we predict
the probability of both classes on the given data point using the mean and variance of the
given cluster. This mean and variance for a given cluster is more accurate since it contains more data points concentrated in a given region and hence the values are more likely to be
closer to the true value. We thus predict the class label for each data point in the test set and
evaluate against the true labels. We obtain the following results when we compare with
various combinations of the model including 2, 4, 8, 16 Gaussian components, both with the
full covariance matrix and the diagonal covariance matrix. A best of 5 runs was taken as the
result for the experiment.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score    

class GMM:
    def __init__(self, train_data,train_label, k, max_iter=1):
        self.k = k
        self.max_iter = int(max_iter)

        self.Train_Data = train_data
        self.Train_Label = train_label
        self.postive_num = 0                                                    
        self.negetive_num = 0                                                   
        postive_data = []                                                       
        negetive_data = [] 
                                                            
        for (data,label) in zip(self.Train_Data,self.Train_Label):
            if label == 1:          
                self.postive_num += 1
                postive_data.append(list(data))
            else:                   
                self.negetive_num += 1
                negetive_data.append(list(data))

        self.pos_means, self.pos_sigma, self.pos_diag_sigma = self.fit(np.asarray(postive_data))
        self.neg_means, self.neg_sigma, self.neg_diag_sigma = self.fit(np.asarray(negetive_data))

    def cluster_assign(self, X):        
        cluster_assignment = {i:[] for i in range(self.k)}
        for point in X:
            cluster_dist =[]
            for cluster in self.mu:
                cluster_dist.append(np.sqrt(np.sum((point-cluster)**2)))
            located_cluster = cluster_dist.index(min(cluster_dist))
            cluster_assignment[located_cluster].append(point)

        groupby_points_previous = {cluster:len(points) for cluster,points in cluster_assignment.items()}        
        return groupby_points_previous,cluster_assignment

    def fit_kmeans(self, X):

        self.sigma = []
        self.diag_sigma = []
        self.weights = []
        self.mu = []
    
        np.random.shuffle(X)        
        random_row = np.random.randint(low=0, high=self.n, size=self.k)
        self.mu = np.asarray([ X[row_index,:] for row_index in random_row ])
        grouped_cluster_prev, assigned_clusters_prev = self.cluster_assign(X)
        
        while True:      
            groupby_points_next, assigned_clusters_new = self.cluster_assign(X)            
            if groupby_points_next == grouped_cluster_prev:
                break                
            else:
                 grouped_cluster_prev, assigned_clusters_prev = groupby_points_next, assigned_clusters_new

        self.weights = np.full( shape=self.shape, fill_value=1/self.k)
        for cluster, points in assigned_clusters_new.items():
            temp = np.cov(np.asarray(points).T)
            self.sigma.append(temp)

        self.mu = np.asarray([ X[row_index,:] for row_index in random_row ])
        self.sigma = np.asarray([ np.cov(X.T) for _ in range(self.k) ])
        self.diag_sigma = np.asarray([ np.diag(np.diag(np.cov(X.T))) for _ in range(self.k) ])

    def initialize(self, X):
        self.shape = X.shape
        self.n, self.m = self.shape
        self.phi = np.full(shape=self.k, fill_value=1/self.k)
        self.fit_kmeans(X) 

    def e_step(self, X):
        self.weights = self.predict_proba(X)
        self.phi = self.weights.mean(axis=0)
    
    def m_step(self, X):
        for i in range(self.k):
            weight = self.weights[:, [i]]
            total_weight = weight.sum()
            self.mu[i] = (X * weight).sum(axis=0) / total_weight
            self.sigma[i] = np.cov(X.T, aweights=(weight/total_weight).flatten(), bias=True)
            self.diag_sigma[i] = np.diag(np.diag(self.sigma[i]))

    def fit(self, X):
        self.initialize(X)
        
        for iteration in range(self.max_iter):
            self.e_step(X)
            self.m_step(X)
        
        return self.mu, self.sigma, self.diag_sigma
            
    def predict_proba(self, X):
        likelihood = np.zeros( (X.shape[0], self.k) )

        for i in range(self.k):
            probab = []
            for x in X:
              probab.append(self.Gaussian(x, self.mu[i], self.sigma[i]))
            probab_np = np.array(probab)
            likelihood[:,i] = probab_np
        
        numerator = likelihood * self.phi
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        weights = numerator / denominator
        return weights
    
    def predict_class(self, X):
        weights = self.predict_proba(X)
        return np.argmax(weights, axis=1)

    def Gaussian(self, x, mean, cov):
        dim = np.shape(cov)[0]
        # Cov measures of the determinant is zero
        covdet = np.linalg.det(cov + np.eye(dim) * 0.001)
        covinv = np.linalg.inv(cov + np.eye(dim) * 0.001)
        xdiff = (x - mean).reshape((1, dim))
        # Probability Density
        prob = 1.0 / (np.power(np.power(2 * np.pi, dim) * np.abs(covdet), 0.5)) * np.exp(-0.5 * xdiff @ covinv @ xdiff.T)[0][0]
        return prob

    def predict(self,test_data, co_var_mat):
        predict_label = []

        pred_class = self.predict_class(test_data)

        for data, data_class in zip(test_data, pred_class):

            # co_var_mat = 0 -> Full covariance matrix
            if co_var_mat == 0:
                class_B_prob = self.Gaussian(data,self.pos_means[data_class],self.pos_sigma[data_class])
                class_A_prob = self.Gaussian(data,self.neg_means[data_class],self.neg_sigma[data_class])

            # co_var_mat != 0 -> Diagonal covariance matrix
            else:
                class_B_prob = self.Gaussian(data,self.pos_means[data_class],self.pos_diag_sigma[data_class])
                class_A_prob = self.Gaussian(data,self.neg_means[data_class],self.neg_diag_sigma[data_class])
              
            if class_B_prob >= class_A_prob:
                predict_label.append(1)
            else:
                predict_label.append(0)
        return predict_label

def run_main():

    for i in range(4):

        gmm = GMM(Xtrain,ytrain, 2**(i+1), 1)

        # TRAIN accuracies.
        # #Predict for full co-variance matrix
        # ytrain_full_pred = gmm.predict(Xtrain, 0)
        # print("\nGMM train accuracy with",2**(i+1)," Gaussian components and full covariance matrix:",accuracy_score(ytrain, ytrain_full_pred))

        # #Predict for diagonal co-variance matrix
        # ytrain_diag_pred = gmm.predict(Xtrain, 1)
        # print("\nGMM train accuracy with",2**(i+1)," Gaussian components and diagonal covariance matrix:",accuracy_score(ytrain,ytrain_diag_pred))

        # TEST accuracies.
        #Predict for full co-variance matrix
        ytest_full_pred = gmm.predict(Xtest, 0)
        print("\nGMM test accuracy with",2**(i+1)," Gaussian components and full covariance matrix:",accuracy_score(ytest, ytest_full_pred))

        #Predict for diagonal co-variance matrix
        ytest_diag_pred = gmm.predict(Xtest, 1)
        print("\nGMM test accuracy with",2**(i+1)," Gaussian components and diagonal covariance matrix:",accuracy_score(ytest,ytest_diag_pred))

if __name__ == '__main__':
    run_main()


GMM test accuracy with 2  Gaussian components and full covariance matrix: 0.7542168674698795

GMM test accuracy with 2  Gaussian components and diagonal covariance matrix: 0.7409638554216867

GMM test accuracy with 4  Gaussian components and full covariance matrix: 0.7674698795180723

GMM test accuracy with 4  Gaussian components and diagonal covariance matrix: 0.6120481927710844

GMM test accuracy with 8  Gaussian components and full covariance matrix: 0.7373493975903614

GMM test accuracy with 8  Gaussian components and diagonal covariance matrix: 0.7168674698795181

GMM test accuracy with 16  Gaussian components and full covariance matrix: 0.8

GMM test accuracy with 16  Gaussian components and diagonal covariance matrix: 0.7843373493975904


As we see in the results, the GMM model performed much better than the multivariate
Gaussian models since they were able to model the data in higher dimensions and thus
compute accurate classifications. The model was also trained for only one epoch for this
experiment for which the GMMs are able to provide sufficiently good results. We also notice
that among all the different number of clusters and thus their Gaussian components, we find
that GMMs performed best with 4 Gaussian components followed by 8, 2, and 16 for this
dataset. Thus, we can conclude that the mixture models are better able to capture the features
of the training data as compared to multivariate Gaussian models and thus give a better
accuracy/ result on the data.